# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import re
import pickle
import pandas as pd 
from sqlalchemy import create_engine
import nltk 
from nltk.tokenize import word_tokenize 
from nltk.corpus import stopwords 
from nltk.stem import WordNetLemmatizer
from sklearn.pipeline import Pipeline 
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.model_selection import train_test_split,  GridSearchCV 
from sklearn.metrics import classification_report


nltk.download(['wordnet', 'punkt', 'stopwords'])



[nltk_data] <urlopen error [Errno 11001] getaddrinfo failed>


False

In [2]:
# load data from database

engine = create_engine('sqlite:///data//disaster_messages_database.db')
df = pd.read_sql_table('disaster_messages_table',engine)

X = df['message']  # Message Column
Y = df.iloc[:, 4:] # Classification label

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    """
    Function: split text into words and return the root form of the words
    Args:
      text(str): the message
    Return:
      lemm(list of str): a list of the root form of the message words
    """
    # Normalize text
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # Tokenize text
    words = word_tokenize(text)
    
    # Remove stop words
    stop = stopwords.words("english")
    words = [t for t in words if t not in stop]
    
    # Lemmatization
    lemm = [WordNetLemmatizer().lemmatize(w) for w in words]
    return lemm

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
# Pipleine 1: Random Forest Classifier

pipeline_rfc = Pipeline([
        ('vect', CountVectorizer(tokenizer = tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf',  MultiOutputClassifier(RandomForestClassifier()))
    ])


# Pipeline 2: Naive Bayes classifier

pipeline_nbc = Pipeline([
        ('vect', CountVectorizer(tokenizer = tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf',  MultiOutputClassifier(MultinomialNB()))
    ])


# Pipleine 3: Adaboost Classifier 

pipeline_ada = Pipeline([
        ('vect', CountVectorizer(tokenizer = tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf',  MultiOutputClassifier(AdaBoostClassifier()))
    ])



### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
# Spiliting data
X_train, X_test, y_train, y_test = train_test_split(X, Y)


In [6]:
# Fit the Random Forest Classifier 
pipeline_rfc.fit(X_train, y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x0000015A8BD18670>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

In [7]:
# Fit the Naive Bayes classifier 
pipeline_nbc.fit(X_train, y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x0000015A8BD18670>)),
                ('tfidf', TfidfTransformer()),
                ('clf', MultiOutputClassifier(estimator=MultinomialNB()))])

In [8]:
# Fit the Adaboost Classifier 
pipeline_ada.fit(X_train, y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x0000015A8BD18670>)),
                ('tfidf', TfidfTransformer()),
                ('clf', MultiOutputClassifier(estimator=AdaBoostClassifier()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [9]:
def plot_scores(y_test, y_pred):
    #Testing the model
    # Printing the classification report for each label
    i = 0
    for col in y_test:
        print('Feature {}: {}'.format(i+1, col))
        print(classification_report(y_test[col], y_pred[:, i]))
        i = i + 1
    accuracy = (y_pred == y_test.values).mean()
    print('The model accuracy is {:.3f}'.format(accuracy))

In [10]:
# Prediction: the Random Forest Classifier  
y_pred = pipeline_rfc.predict(X_test)
plot_scores(y_test, y_pred)


Feature 1: related
              precision    recall  f1-score   support

           0       0.68      0.39      0.50      1511
           1       0.84      0.94      0.89      4994
           2       0.40      0.49      0.44        49

    accuracy                           0.81      6554
   macro avg       0.64      0.61      0.61      6554
weighted avg       0.80      0.81      0.79      6554

Feature 2: request
              precision    recall  f1-score   support

           0       0.91      0.98      0.94      5443
           1       0.84      0.50      0.63      1111

    accuracy                           0.90      6554
   macro avg       0.87      0.74      0.79      6554
weighted avg       0.90      0.90      0.89      6554

Feature 3: offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6523
           1       0.00      0.00      0.00        31

    accuracy                           1.00      6554
   macro avg       

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.98      1.00      0.99      6436
           1       0.00      0.00      0.00       118

    accuracy                           0.98      6554
   macro avg       0.49      0.50      0.50      6554
weighted avg       0.96      0.98      0.97      6554

Feature 9: military
              precision    recall  f1-score   support

           0       0.97      1.00      0.98      6333
           1       0.79      0.05      0.09       221

    accuracy                           0.97      6554
   macro avg       0.88      0.52      0.54      6554
weighted avg       0.96      0.97      0.95      6554

Feature 10: child_alone
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6554

    accuracy                           1.00      6554
   macro avg       1.00      1.00      1.00      6554
weighted avg       1.00      1.00      1.00      6554

Feature 11: water
           

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metr

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6479
           1       0.00      0.00      0.00        75

    accuracy                           0.99      6554
   macro avg       0.49      0.50      0.50      6554
weighted avg       0.98      0.99      0.98      6554

Feature 26: shops
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6527
           1       0.00      0.00      0.00        27

    accuracy                           1.00      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0.99      1.00      0.99      6554

Feature 27: aid_centers
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6474
           1       0.00      0.00      0.00        80

    accuracy                           0.99      6554
   macro avg       0.49      0.50      0.50      6554
weighted avg       0.98      0.9

In [11]:
# Prediction: the Naive Bayes classifier 
y_pred = pipeline_nbc.predict(X_test)
plot_scores(y_test, y_pred)

Feature 1: related
              precision    recall  f1-score   support

           0       0.78      0.09      0.17      1511
           1       0.78      0.99      0.87      4994
           2       0.00      0.00      0.00        49

    accuracy                           0.78      6554
   macro avg       0.52      0.36      0.35      6554
weighted avg       0.77      0.78      0.70      6554

Feature 2: request
              precision    recall  f1-score   support

           0       0.86      0.99      0.92      5443
           1       0.85      0.23      0.36      1111

    accuracy                           0.86      6554
   macro avg       0.86      0.61      0.64      6554
weighted avg       0.86      0.86      0.83      6554

Feature 3: offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6523
           1       0.00      0.00      0.00        31

    accuracy                           1.00      6554
   macro avg       

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metr

              precision    recall  f1-score   support

           0       0.97      1.00      0.99      6369
           1       0.00      0.00      0.00       185

    accuracy                           0.97      6554
   macro avg       0.49      0.50      0.49      6554
weighted avg       0.94      0.97      0.96      6554

Feature 8: security
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      6436
           1       0.00      0.00      0.00       118

    accuracy                           0.98      6554
   macro avg       0.49      0.50      0.50      6554
weighted avg       0.96      0.98      0.97      6554

Feature 9: military
              precision    recall  f1-score   support

           0       0.97      1.00      0.98      6333
           1       0.00      0.00      0.00       221

    accuracy                           0.97      6554
   macro avg       0.48      0.50      0.49      6554
weighted avg       0.93      0.97 

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metr

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6482
           1       0.00      0.00      0.00        72

    accuracy                           0.99      6554
   macro avg       0.49      0.50      0.50      6554
weighted avg       0.98      0.99      0.98      6554

Feature 17: refugees
              precision    recall  f1-score   support

           0       0.97      1.00      0.98      6350
           1       0.00      0.00      0.00       204

    accuracy                           0.97      6554
   macro avg       0.48      0.50      0.49      6554
weighted avg       0.94      0.97      0.95      6554

Feature 18: death
              precision    recall  f1-score   support

           0       0.96      1.00      0.98      6262
           1       0.00      0.00      0.00       292

    accuracy                           0.96      6554
   macro avg       0.48      0.50      0.49      6554
weighted avg       0.91      0.96  

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metr

              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6518
           1       0.00      0.00      0.00        36

    accuracy                           0.99      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0.99      0.99      0.99      6554

Feature 25: hospitals
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6479
           1       0.00      0.00      0.00        75

    accuracy                           0.99      6554
   macro avg       0.49      0.50      0.50      6554
weighted avg       0.98      0.99      0.98      6554

Feature 26: shops
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6527
           1       0.00      0.00      0.00        27

    accuracy                           1.00      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0.99      1.00 

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metr

In [12]:
# Prediction: the Adaboost Classifier 
y_pred = pipeline_ada.predict(X_test)
plot_scores(y_test, y_pred)

Feature 1: related
              precision    recall  f1-score   support

           0       0.62      0.12      0.20      1511
           1       0.78      0.98      0.87      4994
           2       0.57      0.24      0.34        49

    accuracy                           0.77      6554
   macro avg       0.66      0.45      0.47      6554
weighted avg       0.74      0.77      0.71      6554

Feature 2: request
              precision    recall  f1-score   support

           0       0.91      0.96      0.93      5443
           1       0.73      0.53      0.61      1111

    accuracy                           0.89      6554
   macro avg       0.82      0.74      0.77      6554
weighted avg       0.88      0.89      0.88      6554

Feature 3: offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6523
           1       0.18      0.06      0.10        31

    accuracy                           0.99      6554
   macro avg       

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6527
           1       0.11      0.04      0.06        27

    accuracy                           0.99      6554
   macro avg       0.55      0.52      0.53      6554
weighted avg       0.99      0.99      0.99      6554

Feature 27: aid_centers
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6474
           1       0.39      0.09      0.14        80

    accuracy                           0.99      6554
   macro avg       0.69      0.54      0.57      6554
weighted avg       0.98      0.99      0.98      6554

Feature 28: other_infrastructure
              precision    recall  f1-score   support

           0       0.96      0.99      0.97      6257
           1       0.34      0.09      0.14       297

    accuracy                           0.95      6554
   macro avg       0.65      0.54      0.56      6554
weighted avg     

### 6. Improve your model
Use grid search to find better parameters. 

In [13]:
# Show parameters for the pipline
pipeline_rfc.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(tokenizer=<function tokenize at 0x0000015A8BD18670>)),
  ('tfidf', TfidfTransformer()),
  ('clf', MultiOutputClassifier(estimator=RandomForestClassifier()))],
 'verbose': False,
 'vect': CountVectorizer(tokenizer=<function tokenize at 0x0000015A8BD18670>),
 'tfidf': TfidfTransformer(),
 'clf': MultiOutputClassifier(estimator=RandomForestClassifier()),
 'vect__analyzer': 'word',
 'vect__binary': False,
 'vect__decode_error': 'strict',
 'vect__dtype': numpy.int64,
 'vect__encoding': 'utf-8',
 'vect__input': 'content',
 'vect__lowercase': True,
 'vect__max_df': 1.0,
 'vect__max_features': None,
 'vect__min_df': 1,
 'vect__ngram_range': (1, 1),
 'vect__preprocessor': None,
 'vect__stop_words': None,
 'vect__strip_accents': None,
 'vect__token_pattern': '(?u)\\b\\w\\w+\\b',
 'vect__tokenizer': <function __main__.tokenize(text)>,
 'vect__vocabulary': None,
 'tfidf__norm': 'l2',
 'tfidf__smooth_idf': True,
 'tfidf__sublinear_tf': False,


In [14]:
# Show parameters for the pipline
pipeline_ada.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(tokenizer=<function tokenize at 0x0000015A8BD18670>)),
  ('tfidf', TfidfTransformer()),
  ('clf', MultiOutputClassifier(estimator=AdaBoostClassifier()))],
 'verbose': False,
 'vect': CountVectorizer(tokenizer=<function tokenize at 0x0000015A8BD18670>),
 'tfidf': TfidfTransformer(),
 'clf': MultiOutputClassifier(estimator=AdaBoostClassifier()),
 'vect__analyzer': 'word',
 'vect__binary': False,
 'vect__decode_error': 'strict',
 'vect__dtype': numpy.int64,
 'vect__encoding': 'utf-8',
 'vect__input': 'content',
 'vect__lowercase': True,
 'vect__max_df': 1.0,
 'vect__max_features': None,
 'vect__min_df': 1,
 'vect__ngram_range': (1, 1),
 'vect__preprocessor': None,
 'vect__stop_words': None,
 'vect__strip_accents': None,
 'vect__token_pattern': '(?u)\\b\\w\\w+\\b',
 'vect__tokenizer': <function __main__.tokenize(text)>,
 'vect__vocabulary': None,
 'tfidf__norm': 'l2',
 'tfidf__smooth_idf': True,
 'tfidf__sublinear_tf': False,
 'tfidf_

In [15]:
# Using grid search
# Create Grid search parameters for Random Forest Classifier   
parameters_rfc = {
        'tfidf__use_idf': (True, False),
        'clf__estimator__n_estimators': [10, 20]
}

cv_rfc = GridSearchCV(pipeline_rfc, param_grid = parameters_rfc)
cv_rfc

GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x0000015A8BD18670>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier()))]),
             param_grid={'clf__estimator__n_estimators': [10, 20],
                         'tfidf__use_idf': (True, False)})

In [16]:
# Using grid search
# Create Grid search parameters
parameters_ada = {
        'tfidf__use_idf': (True, False),
        'clf__estimator__n_estimators': [50, 60, 70]
}

cv_ada = GridSearchCV(pipeline_ada, param_grid = parameters_ada)
cv_ada

GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x0000015A8BD18670>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=AdaBoostClassifier()))]),
             param_grid={'clf__estimator__n_estimators': [50, 60, 70],
                         'tfidf__use_idf': (True, False)})

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [17]:
# Fit the first tuned model
cv_rfc.fit(X_train, y_train)

GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x0000015A8BD18670>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier()))]),
             param_grid={'clf__estimator__n_estimators': [10, 20],
                         'tfidf__use_idf': (True, False)})

In [18]:
# Fit the second tuned model
cv_ada.fit(X_train, y_train)


GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x0000015A8BD18670>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=AdaBoostClassifier()))]),
             param_grid={'clf__estimator__n_estimators': [50, 60, 70],
                         'tfidf__use_idf': (True, False)})

In [19]:
# Predicting using the first tuned model 
y_pred = cv_rfc.predict(X_test)
plot_scores(y_test, y_pred)

Feature 1: related
              precision    recall  f1-score   support

           0       0.64      0.42      0.51      1511
           1       0.84      0.93      0.88      4994
           2       0.47      0.45      0.46        49

    accuracy                           0.81      6554
   macro avg       0.65      0.60      0.62      6554
weighted avg       0.79      0.81      0.79      6554

Feature 2: request
              precision    recall  f1-score   support

           0       0.91      0.98      0.94      5443
           1       0.83      0.50      0.63      1111

    accuracy                           0.90      6554
   macro avg       0.87      0.74      0.78      6554
weighted avg       0.89      0.90      0.89      6554

Feature 3: offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6523
           1       0.00      0.00      0.00        31

    accuracy                           1.00      6554
   macro avg       

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.97      1.00      0.98      6350
           1       0.50      0.08      0.14       204

    accuracy                           0.97      6554
   macro avg       0.74      0.54      0.56      6554
weighted avg       0.96      0.97      0.96      6554

Feature 18: death
              precision    recall  f1-score   support

           0       0.96      1.00      0.98      6262
           1       0.82      0.14      0.24       292

    accuracy                           0.96      6554
   macro avg       0.89      0.57      0.61      6554
weighted avg       0.96      0.96      0.95      6554

Feature 19: other_aid
              precision    recall  f1-score   support

           0       0.87      1.00      0.93      5657
           1       0.64      0.06      0.10       897

    accuracy                           0.87      6554
   macro avg       0.76      0.53      0.52      6554
weighted avg       0.84      0.87 

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metr

In [20]:
# Predicting using the second tuned model 
y_pred = cv_ada.predict(X_test)
plot_scores(y_test, y_pred)

Feature 1: related
              precision    recall  f1-score   support

           0       0.62      0.07      0.13      1511
           1       0.77      0.99      0.87      4994
           2       0.63      0.24      0.35        49

    accuracy                           0.77      6554
   macro avg       0.68      0.43      0.45      6554
weighted avg       0.74      0.77      0.69      6554

Feature 2: request
              precision    recall  f1-score   support

           0       0.91      0.96      0.93      5443
           1       0.73      0.53      0.61      1111

    accuracy                           0.89      6554
   macro avg       0.82      0.74      0.77      6554
weighted avg       0.88      0.89      0.88      6554

Feature 3: offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6523
           1       0.00      0.00      0.00        31

    accuracy                           0.99      6554
   macro avg       

              precision    recall  f1-score   support

           0       0.86      0.96      0.91      5243
           1       0.72      0.39      0.51      1311

    accuracy                           0.85      6554
   macro avg       0.79      0.68      0.71      6554
weighted avg       0.84      0.85      0.83      6554

The model accuracy is 0.946


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [21]:
# Create a pickle file for the model
file_name = 'classifier.pkl'
with open (file_name, 'wb') as f:
    pickle.dump(cv_ada, f)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.